In [ ]:
#constants that get changed when notebook is autorun
SCOUT_FOLDER_PATH = None

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mpcolors
import numpy as np

import load_mtf_data
import os

In [ ]:
#set matplotlib to display plots in notbook output rather than new window
%matplotlib inline

In [ ]:
#use load_mtf_data python script to get data from the most recent date range subfolder
(interactions, im, subfolder_name) = load_mtf_data.from_most_recent_subfolder(SCOUT_FOLDER_PATH)

In [ ]:
print(im.format, im.size, im.mode)
plt.imshow(im, cmap="gray")

In [ ]:
interactions[0]

We will be plotting interactions by the x,y coordinates of their waypoints.  Matplotlib likes to have coordinates in separate lists for x and y, instead of a single list of x, y tuples.  Thus we will add a new key to each interaction, called "Waypoints-MPL-Format" which stores waypoint coordinates in such a manner.

In [ ]:
for i in interactions: 
    i['Waypoints-MPL-Format'] = list(zip(*i['Waypoints']))
    
#zip(*[(1, 4), (2, 5), (3, 6)]) -> ([1, 2, 3], [4, 5, 6])

In [ ]:
interactions[0]

Next, choose an arbitrary subset of interactions to display.  This is just to get a sense of where the paths are--we don't need to display *everything* as that would get too muddled.

In [ ]:
num_interactions_to_show = 150
interactions_to_show = interactions[:num_interactions_to_show]

We want to color the path of each interaction based on the total duration of the interaction.  We will set a color gradient, cmap, and use norm to assign each color on the gradient to a duration such that the highest color matches with the longest duration of all the paths we are displaying.

In [ ]:
cmap = plt.cm.rainbow
norm = mpcolors.Normalize(vmin=0, vmax=max(i['Duration'] for i in interactions_to_show))

In [ ]:
title = "\n".join([
    str(num_interactions_to_show) + " Randomly Selected Paths",
    "(colored based on interaction time in seconds)",
    "scout: " + os.path.basename(SCOUT_FOLDER_PATH),
    "dates range subfolder: " + subfolder_name
])

In [ ]:
#get axes object on which to plot
ax = plt.axes()

#display the calibration image as background
ax.imshow(im, cmap="gray")

for i in interactions_to_show:
        
    ax.plot(i['Waypoints-MPL-Format'][0],    #x coords
            i['Waypoints-MPL-Format'][1],    #y coords
            alpha=.3,                        #make line somewhat transparent so as not to obscure the background image
            color=cmap(norm(i['Duration']))) #color line based on its duration, using the color map and norm we defined.

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

#make a colorbar of the color scale used:
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, 
             ax=ax, #put it on the axes object ax
             orientation="horizontal", 
             pad=0, 
             shrink=.98
)

ax.set_title(title)

plt.savefig(SCOUT_FOLDER_PATH + os.sep + "paths_" + subfolder_name + ".jpg",
           bbox_inches='tight', dpi=300)